In [10]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt


In [12]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

In [14]:
input_shape = (28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])
x_train = x_train / 255
x_test = x_test / 255


Number of images in x_train 60000
Number of images in x_test 10000


In [17]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

In [18]:

model = Sequential()
model.add(Conv2D(28, kernel_size=(3,3), input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(10,activation=tf.nn.softmax))
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(x=x_train,y=y_train, epochs=50)
model.evaluate(x_test, y_test)
model.save('MNIST-CNN.model')

Epoch 1/50
1875/1875 [==============================] - 22s 11ms/step - loss: 0.6065 - accuracy: 0.8184
Epoch 2/50
1875/1875 [==============================] - 21s 11ms/step - loss: 0.2272 - accuracy: 0.9328
Epoch 3/50
1875/1875 [==============================] - 21s 11ms/step - loss: 0.1640 - accuracy: 0.9509
Epoch 4/50
1875/1875 [==============================] - 21s 11ms/step - loss: 0.1362 - accuracy: 0.9587
Epoch 5/50
1875/1875 [==============================] - 21s 11ms/step - loss: 0.1164 - accuracy: 0.9644
Epoch 6/50
1875/1875 [==============================] - 21s 11ms/step - loss: 0.1050 - accuracy: 0.9681
Epoch 7/50
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0943 - accuracy: 0.9706
Epoch 8/50
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0857 - accuracy: 0.9739
Epoch 9/50
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0800 - accuracy: 0.9749
Epoch 10/50
1875/1875 [==============================] - 21s 11m

## test

In [1]:
from keras.models  import load_model
import numpy as np
import cv2

#load your model
model = load_model("MNIST-CNN.model")



In [2]:
#load your image to recognit image
image = cv2.imread("img.png")
image1 = cv2.imread("img.png")
#perform some basic operation to smooth image
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image = cv2.GaussianBlur(image, (5, 5), 0)

#find threshold image
ret, image = cv2.threshold(image, 90, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

#find contours on image and draw it.
ctrs, hier = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
rects = [cv2.boundingRect(ctr) for ctr in ctrs]
for rect in rects:
    #draw rectangle on image using contours
    cv2.rectangle(image1, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3)
    leng = int(rect[3] * 1.6)
    pt1 = abs(int(rect[1] + rect[3] // 2 - leng // 2))
    pt2 = abs(int(rect[0] + rect[2] // 2 - leng // 2))
    #resize image
    roi = image[pt1:pt1+leng, pt2:pt2+leng]
    roi = cv2.resize(roi,(28, 28), interpolation=cv2.INTER_AREA)

    #reshape your image according to your model
    roi = roi.reshape(-1,28, 28, 1)
    roi = np.array(roi, dtype='float32')
    roi /= 255
    #to perform prediction on your image
    pred_array = model.predict(roi)
    pred_array = np.argmax(pred_array)

    #print result
    print('Result: {0}'.format(pred_array))

    #print text on your image
    cv2.putText(image1, str(pred_array), (rect[0], rect[1]),cv2.FONT_HERSHEY_DUPLEX, 2, (0, 0, 255), 3)
    #show your image
cv2.imshow("Result4",image1)

#save your image
cv2.imwrite("image4.jpg",image1)

cv2.waitKey(0)


Result: 1
Result: 0
Result: 2
Result: 3
Result: 4
Result: 7
Result: 6


-1